### Ensure all requirements are installed from requirements.txt, by running `pip install -r requirements.txt`
### Imprtant: Ensure Cuda enabled GPU is enabled for max performance, otherwise will run on CPU which is orders slower.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
import torchvision
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm
import numpy as np
import os
import pickle
import cv2
import time
import sys

import math
import random
import time


# https://github.com/jorge-pessoa/pytorch-colors
import pytorch_colors
  
from models import RRDB_AutoEncoderSpatial,RRDB_AutoEncoderTemporal
from utils import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Spatial Denoiser
model_spatial = RRDB_AutoEncoderSpatial()
model_spatial.load_state_dict(torch.load('model_spatial_state_dict.pth',map_location=device))
model_spatial.to(device)
model_spatial.eval()

# Temporal Denoiser
model_temporal = RRDB_AutoEncoderTemporal()
model_temporal.load_state_dict(torch.load('model_temporal_state_dict.pth',map_location=device))
model_temporal.to(device)
model_temporal.eval()

## Run this cell to perform the video denoising and saved the results under "Denoised"
## The image seqeunce can then be exported to a video file using the appropriate settings you require, e.g. using FFMPEG

- The image sequence should be stored under "Source/" named "00000.png", "00001.png" and so on
- The output denoised image sequence will be saved under "Denoised/" named "00000.png", "00001.png" and so on




In [ ]:
# The image sequence should be stored under "Source/" named "00000.png", "00001.png" and so on
# The output denoised image sequence will be saved under "Denoised/" named "00000.png", "00001.png" and so on
# Run this cell to perform the video denoising and saved the results under "Denoised"
# The image seqeunce can then be exported to a video file using the appropriate settings you require, e.g. using FFMPEG

def normalize(n,L=5):
    return '0'*(L-len(str(n)))+str(n)

if 'Source' not in os.listdir():
  os.mkdir('Source')
if 'Denoised' not in os.listdir():
  os.mkdir('Denoised')

for file in os.listdir('Denoised'):
  os.remove(os.path.join('Denoised',file))

tot_frames = len(os.listdir('Source'))

for i in tqdm(range(tot_frames)):
    img1 = cv2.imread(os.path.join('Source',normalize(i)+'.png'))
    if i == 0:
        img0 = cv2.imread(os.path.join('Source',normalize(i)+'.png'))
        img2 = cv2.imread(os.path.join('Source',normalize(i+1)+'.png'))
    elif i == tot_frames-1:
        img0 = cv2.imread(os.path.join('Source',normalize(i-1)+'.png'))
        img2 = cv2.imread(os.path.join('Source',normalize(i)+'.png'))
    else:
        img0 = cv2.imread(os.path.join('Source',normalize(i-1)+'.png'))
        img2 = cv2.imread(os.path.join('Source',normalize(i+1)+'.png'))

    #imgdn = denoise_video_frame_patches(img0,img1,img2,model_spatial,model_temporal,patch_size=128,pad=20,BATCH_SIZE=8)
    imgdn = denoise_video_frame(img0,img1,img2,model_spatial,model_temporal)
    cv2.imwrite(os.path.join('Denoised',normalize(i)+'.png'),torch2cv2(imgdn))
